In [6]:
import sys # 시스템 : 쓸수도 있고 안쓸수도 있지만 일단 가져오기
import os  # 시스템 : 쓸수도 있고 안쓸수도 있지만 일단 가져오기

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup    # html 데이터를 전처리 # 아직안씀
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
#from tqdm import tqdm_notebook   # tqdm이전 버전의 사용 방법. 버전이 완전히 바뀌면 아래 코드만을 사용해야 합니다.
from tqdm.notebook import tqdm    # for문 돌릴 때 진행상황을 %게이지로알려줍니다.

In [19]:
original_music_data = pd.read_csv('Music_melon.csv', encoding = 'utf-8')
original_music_data.columns
original_music_data.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace = True)

In [24]:
original_music_title = original_music_data['title']

In [8]:
#Step 1. 크롬 웹브라우저 실행
path = "chromedriver.exe"

driver = webdriver.Chrome(path)
# 사이트 주소는 melon(음원사이트)
driver.get('http://www.melon.com')
time.sleep(0.5)

C:\Users\revol\AppData\Local\Temp/ipykernel_36196/1333834180.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [9]:
# 멜론차트 클릭
driver.find_element_by_css_selector(".menu_bg.menu01").click()
time.sleep(2)

# 일간차트 클릭
driver.find_element_by_css_selector(".menu_chart.m2").click()


C:\Users\revol\AppData\Local\Temp/ipykernel_36196/4150349029.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(".menu_bg.menu01").click()
C:\Users\revol\AppData\Local\Temp/ipykernel_36196/4150349029.py:6: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(".menu_chart.m2").click()


In [ ]:


# 크롤링 코드

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = 100  # 수집할 글 갯수 정하기

for i in range(0,number):
    # 곡정보 더보기 버튼 클릭
    more_info_list = driver.find_elements_by_css_selector(".btn.button_icons.type03.song_info")
    more_info_list[i].click()
    time.sleep(1)
    # 크롤링
    try : 
        music_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성
        time.sleep(1)
        
        # 제목 크롤링
        overlays = ".song_name"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text

        # 가수 크롤링
        overlays = ".artist"                              
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        artist_name = nick.text
        time.sleep(1)

        # 발매날짜 크롤링
        overlays = ".list"                                
        meta_info = driver.find_element_by_css_selector(overlays).text.split('\n')         # datetime
        Album_date = meta_info[3]
        
        # 좋아요 수
        overlays = "d_like_count"
        like_count = driver.find_element_by_id(overlays)
        LIKE = like_count.text
        
        # 앨범이름 크롤링
        overlays = ".list"
        meta_info = driver.find_element_by_css_selector(overlays).text.split('\n')
        Album_name = meta_info[1]
        
        # 현재 곡 장르 크롤링
        overlays = ".list"
        meta_info = driver.find_element_by_css_selector(overlays).text.split('\n')
        Album_Genre = meta_info[5]
        
        # 더보기 버튼 클릭
        driver.find_element_by_css_selector(".button_more.arrow_d").click()
        time.sleep(1)
        
        # 가사 크롤링
        overlays = ".lyric.on"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents
        
        content_list = []
        for content in contents:
            content_list.append(content.text)
 
        content_str = ' '.join(content_list)                         # content_str

        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        music_info['title'] = title
        music_info['artist'] = artist_name
        music_info['Album_date'] = Album_date
        music_info['LIKE'] = LIKE
        music_info['Album_name'] = Album_name
        music_info['Album_Genre'] = Album_Genre
        music_info['Lyric'] = content_str
        time.sleep(1)
        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = music_info
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 뒤로가기
        driver.back()  
    
    # 에러처리 : 에러나면 뒤로가기하고 다음 글(i+1)로 이동합니다.
    except:
        driver.back()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기  - 에러날 경우 데이터 손실 위험 감소
    if i == 30 or 50 or 80:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("Music_melon.xlsx", encoding='utf-8-sig')
        time.sleep(2)

# 저장하기
result_df.to_excel("Music_melon.xlsx", encoding='utf-8-sig')
time.sleep(2)
driver.quit()

print('수집한 글 갯수: ', len(dict))

In [11]:
chart = driver.page_source
soup = BeautifulSoup(chart, 'html.parser')

In [25]:
update_music_title = []
update_title_path = soup.select('tr td div.wrap div.wrap_song_info div.ellipsis.rank01 span a')

for i in range(len(update_title_path)):
    update_music_title.append(update_title_path[i].text)
    
update_music_title = pd.Series(update_music_title)
type(original_music_title) == type(update_music_title)

True

In [32]:
All_music = pd.concat([original_music_title, update_music_title]).drop_duplicates()
update_music_index = All_music[len(original_music_data):].index
len(update_music_index)

13

In [33]:
print(update_music_index)

Int64Index([0, 2, 12, 33, 41, 49, 75, 81, 89, 90, 92, 94, 99], dtype='int64')


In [35]:


# 크롤링 코드

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = len(update_music_index)  # 수집할 글 갯수 정하기

for i in update_music_index:
    # 곡정보 더보기 버튼 클릭
    more_info_list = driver.find_elements_by_css_selector(".btn.button_icons.type03.song_info")
    more_info_list[i].click()
    time.sleep(1)
    # 크롤링
    try : 
        music_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성
        time.sleep(1)
        
        # 제목 크롤링
        overlays = ".song_name"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text

        # 가수 크롤링
        overlays = ".artist"                              
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        artist_name = nick.text
        time.sleep(1)

        # 발매날짜 크롤링
        overlays = ".list"                                
        meta_info = driver.find_element_by_css_selector(overlays).text.split('\n')         # datetime
        Album_date = meta_info[3]
        
        # 좋아요 수
        overlays = "d_like_count"
        like_count = driver.find_element_by_id(overlays)
        LIKE = like_count.text
        
        # 앨범이름 크롤링
        overlays = ".list"
        meta_info = driver.find_element_by_css_selector(overlays).text.split('\n')
        Album_name = meta_info[1]
        
        # 현재 곡 장르 크롤링
        overlays = ".list"
        meta_info = driver.find_element_by_css_selector(overlays).text.split('\n')
        Album_Genre = meta_info[5]
        
        # 더보기 버튼 클릭
        driver.find_element_by_css_selector(".button_more.arrow_d").click()
        time.sleep(1)
        
        # 가사 크롤링
        overlays = ".lyric.on"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents
        
        content_list = []
        for content in contents:
            content_list.append(content.text)
 
        content_str = ' '.join(content_list)                         # content_str

        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        music_info['title'] = title
        music_info['artist'] = artist_name
        music_info['Album_date'] = Album_date
        music_info['LIKE'] = LIKE
        music_info['Album_name'] = Album_name
        music_info['Album_Genre'] = Album_Genre
        music_info['Lyric'] = content_str
        time.sleep(1)
        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = music_info
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 뒤로가기
        driver.back()  
    
    # 에러처리 : 에러나면 뒤로가기하고 다음 글(i+1)로 이동합니다.
    except:
        driver.back()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기  - 에러날 경우 데이터 손실 위험 감소
    if i % 4 == 2:
        # 판다스로 만들기
        import pandas as pd
        new_music_data = pd.DataFrame.from_dict(dict, 'index')

        time.sleep(2)

# 저장하기
update_music_data = pd.concat([original_music_data, new_music_data])
update_music_data.to_excel("new_music.xlsx", encoding='utf-8-sig')
time.sleep(2)
driver.quit()

print('수집한 글 갯수: ', len(dict))

C:\Users\revol\AppData\Local\Temp/ipykernel_36196/750156570.py:9: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  more_info_list = driver.find_elements_by_css_selector(".btn.button_icons.type03.song_info")
C:\Users\revol\AppData\Local\Temp/ipykernel_36196/750156570.py:19: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  tit = driver.find_element_by_css_selector(overlays)          # title
C:\Users\revol\AppData\Local\Temp/ipykernel_36196/750156570.py:24: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  nick = driver.find_element_by_css_selector(overlays)         # nickname
C:\Users\revol\AppData\Local\Temp/ipykernel_36196/750156570.py:30: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_elem

2 LOVE DIVE


KeyboardInterrupt: 

In [36]:
# 저장하기
update_music_data = pd.concat([original_music_data, new_music_data])
update_music_data.to_excel("new_music.xlsx", encoding='utf-8-sig')
time.sleep(2)
driver.quit()

print('수집한 글 갯수: ', len(dict))

수집한 글 갯수:  1
